# StrideStats: Activity Maps and Locations

This notebook focuses on geographic data. We'll use the coordinate and polyline information to visualize where you've been active.

In [2]:
import pandas as pd
import folium
from folium.plugins import HeatMap
import polyline
from pathlib import Path

# Setting default map location (will update based on data)
CENTER_START = [0, 0]

## 1. Load and Prepare Data
We load the processed dataset and filter out activities that don't have coordinate data (e.g., manual entries or indoor activities).

In [3]:
data_path = Path("../data/processed/activities.parquet")
if not data_path.exists():
    print("Error: activities.parquet not found.")
else:
    df = pd.read_parquet(data_path)
    
    # Filter for activities with valid coordinates
    df_map = df[df['start_lat'].notna() & df['start_lng'].notna()].copy()
    
    print(f"Loaded {len(df)} activities. {len(df_map)} have valid location data.")

Loaded 2386 activities. 2089 have valid location data.


## 2. Global Activity Heatmap
Where do your activities usually start? This heatmap shows the concentration of your activity starting points.

In [4]:
if 'df_map' in locals() and not df_map.empty:
    # Center the map on the median location
    map_center = [df_map['start_lat'].median(), df_map['start_lng'].median()]
    m = folium.Map(location=map_center, zoom_start=12, tiles='CartoDB positron')
    
    # Prepare heatmap data
    heat_data = [[row['start_lat'], row['start_lng']] for index, row in df_map.iterrows()]
    
    # Add heatmap layer
    HeatMap(heat_data).add_to(m)
    
    display(m)
else:
    print("No map data available.")

## 3. Visualize a Specific Route
We can use the `map_polyline` field to reconstruct and plot the actual path an activity took.

In [5]:
if 'df_map' in locals() and not df_map.empty:
    # Pick a recent activity with a polyline
    route_df = df_map[df_map['map_polyline'].notna()]
    
    if not route_df.empty:
        sample_activity = route_df.iloc[0]
        print(f"Plotting: {sample_activity['name']} ({sample_activity['start_date']})")
        
        # Decode polyline
        points = polyline.decode(sample_activity['map_polyline'])
        
        # Start map at the start point
        m_route = folium.Map(location=points[0], zoom_start=14)
        
        # Draw the line
        folium.PolyLine(points, color="blue", weight=2.5, opacity=1).add_to(m_route)
        
        display(m_route)
    else:
        print("No activity found with route polyline data.")

Plotting: Morning Rainy Run 🌧️ (2022-11-08 14:37:42+00:00)
